# Imports

In [1]:
from src.inspector_git import IGLogReader, GitLogDTO
from src.jira_miner.models import JsonFileFormatJira
from src.github_miner import JsonFileFormatGithub
import json
from pathlib import Path
from pprint import pprint
import re
from datetime import datetime
import pickle
from graph import *

# Load data from SERIALIZED formats

In [2]:
# JSON
path_jira = "/home/vortex/Work/BachelorThesis/Vortex/test-input/jira-miner/ZEPPELIN-detailed-issues.json"
path_github = "/home/vortex/Work/BachelorThesis/Vortex/test-input/github-miner/githubProject.json"
# IGLOG
path_inspector_git = "/home/vortex/Work/BachelorThesis/Vortex/test-input/inspector-git/zeppelin.iglog"



def load_from_json(model_cls, file_path: str):
    """Load JSON from a given path and validate it with the provided model class."""
    file_path = Path(file_path)

    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    return model_cls.model_validate(data)

def load_jsons():
    jira_data_loaded = load_from_json(JsonFileFormatJira, path_jira)
    github_data_loaded = load_from_json(JsonFileFormatGithub, path_github)
    return jira_data_loaded, github_data_loaded

ig_log_reader = IGLogReader()

jira_data, github_data = load_jsons()
inspector_git_data = ig_log_reader.read(path_inspector_git)

In [3]:
import importlib, graph
importlib.reload(graph)
from graph import *

graph = Graph()

graph.add_inspector_git_data(inspector_git_data)
graph.add_jira_data(jira_data)
graph.add_github_data(github_data)

graph.link_issues_with_git_commits()
graph.link_pull_request_with_issue(jira_data)
graph.link_pull_requests_with_git_commits()

print(graph.summary())

Indexing is 6186
~~~~ Graph summary ~~~~
commits: 5512
git_users: 602
files: 6185

issue_statuses: 48
issue_types: 7
issue_status_categories: 3
jira_users: 2008
issues: 6202

pull_requests: 5022
git_hub_users: 690
git_hub_commits: 17869

nodes: 44148
edges: 246520



In [5]:
graph.files = []
graph.issues = []
graph.save()

RecursionError: maximum recursion depth exceeded while pickling an object

In [5]:
fisiere = [f.path for f in graph.files]
strings_sorted = sorted(fisiere)
# Save to file
with open("recursive.txt", "w") as f:
    for item in strings_sorted:
        f.write(item + "\n")

In [38]:
nr  =0
for file in graph.files:
    if file.path == "/dev/null":
        nr += 1

print(nr)

2270
